In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
bookings = pd.read_csv(r"C:\Users\payoj.jain/Documents/projects/Europe Revenue estimation/data/Booking_level_data.csv")

In [3]:
bookings.head()

,ACCOMMODATION_CODE,BOOKING_ID,BOOKING_STATUS,BOOKING_STATUS_DESCRIPTION,arrivaldate,departuredate,bookingdate,gbv,no_persons,no_child,no_adults
0,AT-5505-11,576245314,BOOKING,BOOKING,7/27/2019,8/3/2019,12/3/2018,2710.75,27,0,24
1,AT-5505-11,76234292,CANCELLED,CANCELLATION (NO PAYMENT),10/5/2019,10/12/2019,11/14/2018,0.00,19,10,9
2,AT-5505-11,1UHGKQGJ,BOOKING,BOOKING,10/5/2019,10/12/2019,8/21/2019,1839.04,23,12,11
3,AT-5505-11,776298327,CANCELLED,CANCELLATION WITHOUT INSURANCE,10/12/2019,10/19/2019,1/28/2019,567.00,24,17,7
4,AT-5541-33,76052288,BOOKING,BOOKING,3/2/2019,3/9/2019,3/7/2018,2288.59,19,6,11


In [4]:
bookings['arrivaldate'] = pd.to_datetime(bookings['arrivaldate'])
bookings['departuredate'] = pd.to_datetime(bookings['departuredate'])
bookings['bookingdate'] = pd.to_datetime(bookings['bookingdate'])

In [25]:
date_to_week = pd.read_csv(r"C:\Users\payoj.jain/Documents/projects/Europe Revenue estimation/data/weeks_start_date.csv")

In [26]:
date_to_week

,YEAR,WEEK,WK_START
0,2017,20,2017-05-13
1,2017,21,2017-05-20
2,2017,22,2017-05-27
3,2017,23,2017-06-03
4,2017,24,2017-06-10
...,...,...,...
185,2020,49,2020-11-28
186,2020,50,2020-12-05
187,2020,51,2020-12-12
188,2020,52,2020-12-19


In [28]:
date_to_week['WK_START'] = pd.to_datetime(date_to_week['WK_START'])

In [29]:
date_to_week["WK_END"]= date_to_week["WK_START"] + datetime.timedelta(7)

In [100]:
date_to_week.head()

,YEAR,WEEK,WK_START,WK_END
0,2017,20,2017-05-13,2017-05-20
1,2017,21,2017-05-20,2017-05-27
2,2017,22,2017-05-27,2017-06-03
3,2017,23,2017-06-03,2017-06-10
4,2017,24,2017-06-10,2017-06-17


In [9]:
DE_bookings = bookings[(bookings["ACCOMMODATION_CODE"].str.contains("DE"))&(bookings["BOOKING_STATUS"]=="BOOKING")]

In [10]:
DE_bookings.head()

,ACCOMMODATION_CODE,BOOKING_ID,BOOKING_STATUS,BOOKING_STATUS_DESCRIPTION,arrivaldate,departuredate,bookingdate,gbv,no_persons,no_child,no_adults
5079,DE-00014-45,276245097,BOOKING,BOOKING,2019-05-29,2019-06-02,2018-12-03,1251.66,17,9,8
5080,DE-00014-45,976211446,BOOKING,BOOKING,2019-07-14,2019-07-21,2018-10-10,2457.10,19,5,14
5081,DE-00014-45,876136955,BOOKING,BOOKING,2018-08-10,2018-08-16,2018-07-03,1865.63,19,4,15
5082,DE-00016-24,876202668,BOOKING,BOOKING,2019-10-03,2019-10-06,2018-09-27,1438.96,24,4,20
5083,DE-00016-24,176212282,BOOKING,BOOKING,2019-09-13,2019-09-15,2018-10-11,1101.42,34,8,22


In [12]:
# Create a day wise booking dataframe. Converts booking level data into day wise. 
day_wise_bookings = pd.DataFrame(columns=["ACCOMMODATION_CODE", "BOOKING_ID", "arrivaldate", "departuredate","bookingdate", "date", "revenue"])
count = 0
for booking in DE_bookings.values:
    if(count%10000==0):
        print(count)
    count+= 1
    days_of_booking = int(((booking[5]-booking[4]) / np.timedelta64(1, 'D')))
    rev_per_day = booking[7]/days_of_booking
    for i in range(days_of_booking):
        date = booking[4] + datetime.timedelta(i)
        booking_df = pd.DataFrame([(booking[0], booking[1], booking[4], booking[5], booking[6], date, rev_per_day)], columns=["ACCOMMODATION_CODE", "BOOKING_ID", "arrivaldate", "departuredate","bookingdate", "date", "revenue"])
        day_wise_bookings = day_wise_bookings.append(booking_df)

0
10000
20000
30000
40000
50000
60000


In [17]:
day_wise_bookings = day_wise_bookings.drop(["index"], axis=1)

In [18]:
day_wise_bookings

,ACCOMMODATION_CODE,BOOKING_ID,arrivaldate,departuredate,bookingdate,date,revenue
0,DE-00014-45,276245097,2019-05-29,2019-06-02,2018-12-03,2019-05-29,312.915000
1,DE-00014-45,276245097,2019-05-29,2019-06-02,2018-12-03,2019-05-30,312.915000
2,DE-00014-45,276245097,2019-05-29,2019-06-02,2018-12-03,2019-05-31,312.915000
3,DE-00014-45,276245097,2019-05-29,2019-06-02,2018-12-03,2019-06-01,312.915000
4,DE-00014-45,976211446,2019-07-14,2019-07-21,2018-10-10,2019-07-14,351.014286
...,...,...,...,...,...,...,...
342874,DE-59964-49,576125398,2019-03-04,2019-03-08,2018-06-18,2019-03-05,244.460000
342875,DE-59964-49,576125398,2019-03-04,2019-03-08,2018-06-18,2019-03-06,244.460000
342876,DE-59964-49,576125398,2019-03-04,2019-03-08,2018-06-18,2019-03-07,244.460000
342877,DE-93462-01,876078548,2018-05-25,2018-05-27,2018-04-12,2018-05-25,488.005000


In [31]:
# Create day wise week dataframe. Converts week level dates to each date 
day_wise_week = pd.DataFrame(columns=["year", "week", "week_start", "week_end","date"])
count = 0
for week in date_to_week.values:
    if(count%365==0):
        print(count)
    days_of_booking = int(((week[3]-week[2]) / np.timedelta64(1, 'D')))
#     rev_per_day = booking[7]/days_of_booking
    for i in range(days_of_booking):
        count+= 1
        date = week[2] + datetime.timedelta(i)
        week_df = pd.DataFrame([(week[0], week[1], week[2], week[3], date)], columns=["year", "week", "week_start", "week_end","date"])
        day_wise_week = day_wise_week.append(week_df)

0


In [32]:
day_wise_week

,year,week,week_start,week_end,date
0,2017,20,2017-05-13,2017-05-20,2017-05-13
0,2017,20,2017-05-13,2017-05-20,2017-05-14
0,2017,20,2017-05-13,2017-05-20,2017-05-15
0,2017,20,2017-05-13,2017-05-20,2017-05-16
0,2017,20,2017-05-13,2017-05-20,2017-05-17
...,...,...,...,...,...
0,2020,53,2020-12-26,2021-01-02,2020-12-28
0,2020,53,2020-12-26,2021-01-02,2020-12-29
0,2020,53,2020-12-26,2021-01-02,2020-12-30
0,2020,53,2020-12-26,2021-01-02,2020-12-31


In [35]:
day_wise_week = day_wise_week.reset_index().drop(["index"], axis=1)

In [37]:
day_wise_week = day_wise_week.drop(["level_0"], axis=1)

In [38]:
day_wise_week

,year,week,week_start,week_end,date
0,2017,20,2017-05-13,2017-05-20,2017-05-13
1,2017,20,2017-05-13,2017-05-20,2017-05-14
2,2017,20,2017-05-13,2017-05-20,2017-05-15
3,2017,20,2017-05-13,2017-05-20,2017-05-16
4,2017,20,2017-05-13,2017-05-20,2017-05-17
...,...,...,...,...,...
1325,2020,53,2020-12-26,2021-01-02,2020-12-28
1326,2020,53,2020-12-26,2021-01-02,2020-12-29
1327,2020,53,2020-12-26,2021-01-02,2020-12-30
1328,2020,53,2020-12-26,2021-01-02,2020-12-31


In [126]:
DE_day_wise_booking = day_wise_bookings.merge(day_wise_week, on="date", how="left")

In [127]:
DE_day_wise_booking.head()

,ACCOMMODATION_CODE,BOOKING_ID,arrivaldate,departuredate,bookingdate,date,revenue,year,week,week_start,week_end
0,DE-00014-45,276245097,2019-05-29,2019-06-02,2018-12-03,2019-05-29,312.915000,2019,22,2019-05-25,2019-06-01
1,DE-00014-45,276245097,2019-05-29,2019-06-02,2018-12-03,2019-05-30,312.915000,2019,22,2019-05-25,2019-06-01
2,DE-00014-45,276245097,2019-05-29,2019-06-02,2018-12-03,2019-05-31,312.915000,2019,22,2019-05-25,2019-06-01
3,DE-00014-45,276245097,2019-05-29,2019-06-02,2018-12-03,2019-06-01,312.915000,2019,23,2019-06-01,2019-06-08
4,DE-00014-45,976211446,2019-07-14,2019-07-21,2018-10-10,2019-07-14,351.014286,2019,29,2019-07-13,2019-07-20


In [128]:
DE_day_wise_booking[(DE_day_wise_booking["ACCOMMODATION_CODE"]=="DE-00013-53")&(DE_day_wise_booking["year"]==2018)&(DE_day_wise_booking["week"]==1)]

,ACCOMMODATION_CODE,BOOKING_ID,arrivaldate,departuredate,bookingdate,date,revenue,year,week,week_start,week_end
231848,DE-00013-53,75986951,2018-01-04,2018-01-07,2018-01-03,2018-01-04,83.12,2018,1,2017-12-30,2018-01-06
231849,DE-00013-53,75986951,2018-01-04,2018-01-07,2018-01-03,2018-01-05,83.12,2018,1,2017-12-30,2018-01-06


In [146]:
DE_weekly_revenue = DE_day_wise_booking.groupby(["year","week","ACCOMMODATION_CODE"]).sum()

In [147]:
DE_weekly_revenue = DE_weekly_revenue.reset_index()

In [157]:
DE_weekly_revenue[(DE_weekly_revenue["ACCOMMODATION_CODE"]=="DE-00014-45")&(DE_weekly_revenue["week"]==23)]

,year,week,ACCOMMODATION_CODE,revenue
45456,2019,23,DE-00014-45,625.225


In [158]:
DE_occupancy = DE_day_wise_booking.groupby(["year","week","ACCOMMODATION_CODE"]).count()["date"]

In [159]:
DE_occupancy = occupancy.reset_index()

In [160]:
DE_occupancy = occupancy.rename(columns={"date":"no_days_booked"})

In [161]:
DE_occupancy["occupancy"] = DE_occupancy["no_days_booked"]/7

In [162]:
DE_occupancy[DE_occupancy["ACCOMMODATION_CODE"]=="DE-00014-45"]

,year,week,ACCOMMODATION_CODE,no_days_booked,occupancy
9360,2018,24,DE-00014-45,5,0.714286
10068,2018,25,DE-00014-45,1,0.142857
12576,2018,28,DE-00014-45,5,0.714286
13647,2018,29,DE-00014-45,2,0.285714
16221,2018,31,DE-00014-45,3,0.428571
17599,2018,32,DE-00014-45,2,0.285714
19024,2018,33,DE-00014-45,5,0.714286
22464,2018,36,DE-00014-45,1,0.142857
23396,2018,37,DE-00014-45,2,0.285714
24351,2018,38,DE-00014-45,1,0.142857


In [163]:
DE_day_wise_booking.head()

,ACCOMMODATION_CODE,BOOKING_ID,arrivaldate,departuredate,bookingdate,date,revenue,year,week,week_start,week_end
0,DE-00014-45,276245097,2019-05-29,2019-06-02,2018-12-03,2019-05-29,312.915000,2019,22,2019-05-25,2019-06-01
1,DE-00014-45,276245097,2019-05-29,2019-06-02,2018-12-03,2019-05-30,312.915000,2019,22,2019-05-25,2019-06-01
2,DE-00014-45,276245097,2019-05-29,2019-06-02,2018-12-03,2019-05-31,312.915000,2019,22,2019-05-25,2019-06-01
3,DE-00014-45,276245097,2019-05-29,2019-06-02,2018-12-03,2019-06-01,312.915000,2019,23,2019-06-01,2019-06-08
4,DE-00014-45,976211446,2019-07-14,2019-07-21,2018-10-10,2019-07-14,351.014286,2019,29,2019-07-13,2019-07-20


In [164]:
DE_weekly_revenue = DE_weekly_revenue.merge(DE_occupancy, on=["year", "week", "ACCOMMODATION_CODE"], how="left")

In [165]:
DE_weekly_revenue

,year,week,ACCOMMODATION_CODE,revenue,no_days_booked,occupancy
0,2018,1,DE-00008-46,167.050000,1,0.142857
1,2018,1,DE-00013-53,166.240000,2,0.285714
2,2018,1,DE-06493-11,68.073333,1,0.142857
3,2018,1,DE-09456-01,73.666667,1,0.142857
4,2018,1,DE-54539-01,376.020000,1,0.142857
...,...,...,...,...,...,...
76238,2020,2,DE-96369-05,60.833333,1,0.142857
76239,2020,2,DE-98553-03,54.104286,1,0.142857
76240,2020,2,DE-98666-04,95.883333,1,0.142857
76241,2020,2,DE-98667-17,87.900000,1,0.142857


In [166]:
DE_weekly_revenue[DE_weekly_revenue["ACCOMMODATION_CODE"]=="DE-00014-45"]

,year,week,ACCOMMODATION_CODE,revenue,no_days_booked,occupancy
9360,2018,24,DE-00014-45,1617.725000,5,0.714286
10068,2018,25,DE-00014-45,332.305000,1,0.142857
12576,2018,28,DE-00014-45,1711.807143,5,0.714286
13647,2018,29,DE-00014-45,684.722857,2,0.285714
16221,2018,31,DE-00014-45,852.675000,3,0.428571
17599,2018,32,DE-00014-45,595.163333,2,0.285714
19024,2018,33,DE-00014-45,1554.691667,5,0.714286
22464,2018,36,DE-00014-45,288.030000,1,0.142857
23396,2018,37,DE-00014-45,614.200000,2,0.285714
24351,2018,38,DE-00014-45,326.170000,1,0.142857


In [167]:
accommodation_df = pd.read_csv(r"C:\Users\payoj.jain/Documents/projects/Europe Revenue estimation/data/X_data.csv")

In [168]:
DE_properties = accommodation_df[accommodation_df["COUNTRY"] == "DE"]

In [169]:
DE_properties = DE_properties.drop(["Unnamed: 0"], axis=1)

In [170]:
model_data = DE_weekly_revenue.merge(DE_properties, on="ACCOMMODATION_CODE", how="right")

In [171]:
model_data = model_data[model_data["revenue"].notnull()]

In [172]:
model_data.head()

,year,week,ACCOMMODATION_CODE,revenue,no_days_booked,occupancy,ACCOMMODATION_TYPE,ACCO_DESCRIPTION,NUMBER_OF_PERSONS,BEDROOM_COUNT,...,distance_from_coast,nearest_coastal_longitude,nearest_coastal_latitude,COUNTRY,price,bookings_count,weeks in 2017,weeks in 2018,weeks in 2019,weeks in 2020
0,2018.0,1.0,DE-00008-46,167.05,1.0,0.142857,2,Holiday home,8,4,...,25.708268,11.499003,54.030168,DE,166.992198,53.0,0.286,41.002,41.717,0.0
1,2018.0,2.0,DE-00008-46,167.05,1.0,0.142857,2,Holiday home,8,4,...,25.708268,11.499003,54.030168,DE,166.992198,53.0,0.286,41.002,41.717,0.0
2,2018.0,8.0,DE-00008-46,167.05,1.0,0.142857,2,Holiday home,8,4,...,25.708268,11.499003,54.030168,DE,166.992198,53.0,0.286,41.002,41.717,0.0
3,2018.0,9.0,DE-00008-46,167.05,1.0,0.142857,2,Holiday home,8,4,...,25.708268,11.499003,54.030168,DE,166.992198,53.0,0.286,41.002,41.717,0.0
4,2018.0,11.0,DE-00008-46,167.05,1.0,0.142857,2,Holiday home,8,4,...,25.708268,11.499003,54.030168,DE,166.992198,53.0,0.286,41.002,41.717,0.0


In [193]:
avail_df = pd.read_csv(r"C:\Users\payoj.jain/Documents/projects/Europe Revenue estimation/data/week_wise_availability.csv")

In [194]:
avail_df = avail_df.rename(columns={"accommodation_code": "ACCOMMODATION_CODE"})

In [195]:
de_avail_df = avail_df[avail_df["ACCOMMODATION_CODE"].str.contains("DE")]

In [196]:
X = de_avail_df.merge(model_data, on=["year","week","ACCOMMODATION_CODE"], how="right")

In [197]:
X

,ACCOMMODATION_CODE,week,year,availablity_fin,revenue,no_days_booked,occupancy,ACCOMMODATION_TYPE,ACCO_DESCRIPTION,NUMBER_OF_PERSONS,...,distance_from_coast,nearest_coastal_longitude,nearest_coastal_latitude,COUNTRY,price,bookings_count,weeks in 2017,weeks in 2018,weeks in 2019,weeks in 2020
0,DE-82442-01,1,2020,0.0,345.320000,5.0,0.714286,1,Apartment,2,...,NaN,NaN,NaN,DE,65.308707,24.0,1.0,47.859,43.431,0.0
1,DE-98711-08,1,2019,1.0,167.265000,3.0,0.428571,1,Apartment,3,...,NaN,NaN,NaN,DE,55.518120,45.0,1.0,48.857,47.857,0.0
2,DE-78112-01,1,2020,0.0,721.890000,7.0,1.000000,2,Holiday home,4,...,NaN,NaN,NaN,DE,78.156470,47.0,1.0,52.000,51.000,0.0
3,DE-79875-12,1,2020,0.0,278.992000,4.0,0.571429,1,Apartment,2,...,NaN,NaN,NaN,DE,64.899046,17.0,1.0,52.001,51.000,0.0
4,DE-82396-01,1,2019,1.0,395.040000,2.0,0.285714,2,Holiday home,2,...,NaN,NaN,NaN,DE,190.079778,8.0,1.0,52.002,51.001,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64023,DE-54552-87,31,2019,NaN,534.242857,5.0,0.714286,1,Apartment,2,...,NaN,NaN,NaN,DE,106.330714,2.0,NaN,NaN,NaN,NaN
64024,DE-54552-87,32,2019,NaN,319.510000,3.0,0.428571,1,Apartment,2,...,NaN,NaN,NaN,DE,106.330714,2.0,NaN,NaN,NaN,NaN
64025,DE-54552-87,33,2019,NaN,634.877143,6.0,0.857143,1,Apartment,2,...,NaN,NaN,NaN,DE,106.330714,2.0,NaN,NaN,NaN,NaN
64026,DE-54552-93,33,2019,NaN,551.594286,4.0,0.571429,1,Apartment,6,...,NaN,NaN,NaN,DE,137.898571,1.0,NaN,NaN,NaN,NaN


In [210]:
X[(X['availablity_fin'] == 0)&(X["week"] < 52)]

,ACCOMMODATION_CODE,week,year,availablity_fin,revenue,no_days_booked,occupancy,ACCOMMODATION_TYPE,ACCO_DESCRIPTION,NUMBER_OF_PERSONS,...,distance_from_coast,nearest_coastal_longitude,nearest_coastal_latitude,COUNTRY,price,bookings_count,weeks in 2017,weeks in 2018,weeks in 2019,weeks in 2020
0,DE-82442-01,1,2020,0.0,345.320000,5.0,0.714286,1,Apartment,2,...,NaN,NaN,NaN,DE,65.308707,24.0,1.0,47.859,43.431,0.0
2,DE-78112-01,1,2020,0.0,721.890000,7.0,1.000000,2,Holiday home,4,...,NaN,NaN,NaN,DE,78.156470,47.0,1.0,52.000,51.000,0.0
3,DE-79875-12,1,2020,0.0,278.992000,4.0,0.571429,1,Apartment,2,...,NaN,NaN,NaN,DE,64.899046,17.0,1.0,52.001,51.000,0.0
5,DE-99848-10,1,2020,0.0,420.175000,7.0,1.000000,1,Apartment,4,...,NaN,NaN,NaN,DE,61.148393,18.0,1.0,52.001,50.430,0.0
6,DE-59955-192,1,2020,0.0,0.000000,7.0,1.000000,1,Apartment,4,...,NaN,NaN,NaN,DE,70.076786,24.0,0.0,45.286,33.000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61432,DE-54298-04,49,2019,0.0,0.000000,6.0,0.857143,1,Apartment,6,...,NaN,NaN,NaN,DE,76.296116,51.0,1.0,52.000,46.859,0.0
61889,DE-54298-04,50,2019,0.0,0.000000,7.0,1.000000,1,Apartment,6,...,NaN,NaN,NaN,DE,76.296116,51.0,1.0,52.000,46.859,0.0
62277,DE-06493-25,51,2019,0.0,85.534286,1.0,0.142857,2,Holiday home,4,...,NaN,NaN,NaN,DE,106.556926,15.0,0.0,38.002,50.000,0.0
62455,DE-54298-01,51,2018,0.0,0.000000,7.0,1.000000,2,Holiday home,5,...,NaN,NaN,NaN,DE,70.855163,36.0,0.0,25.857,29.000,0.0


In [183]:
avail_df.query('ACCOMMODATION_CODE == "DE-94234-16"')

,ACCOMMODATION_CODE,week,year,availability_fin
1454,DE-94234-16,1,2018,1.000
5918,DE-94234-16,1,2020,0.000
7356,DE-94234-16,1,2019,1.000
10310,DE-94234-16,2,2019,0.143
15215,DE-94234-16,2,2018,1.000
...,...,...,...,...
499664,DE-94234-16,51,2018,1.000
501175,DE-94234-16,51,2019,1.000
511179,DE-94234-16,52,2018,1.000
512132,DE-94234-16,52,2017,1.000


In [209]:
DE_day_wise_booking.query("ACCOMMODATION_CODE == 'DE-54298-01' & week==1")

,ACCOMMODATION_CODE,BOOKING_ID,arrivaldate,departuredate,bookingdate,date,revenue,year,week,week_start,week_end


In [207]:
X.to_csv(r"C:\Users\payoj.jain/Documents/projects/Europe Revenue estimation/data/DE_weekly_revenue_dataset.csv")